In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import pendulum
import requests
import matplotlib.pyplot as plt

In [ ]:
def select_point(ds, lat, lon):
    # First, find the index of the grid point nearest a specific lat/lon.   
    abslat = np.abs(ds.XLAT[0]-lat)
    abslon = np.abs(ds.XLONG[0]-lon)
    c = np.maximum(abslon, abslat)

    ([xloc], [yloc]) = np.where(c == np.min(c))

    # Now I can use that index location to get the values at the x/y diminsion
    return ds.sel(south_north=xloc, west_east=yloc)

In [ ]:
filename = "data/wrfout_d01_2022-07-17_00:00:00"

In [ ]:
ds = xr.load_dataset(filename, engine="netcdf4")

start = pendulum.datetime(2022, 7, 17)
end = start.add(hours=3*4)

period = pendulum.period(start, end)

ds = ds.assign_coords({"Time": list(period.range("hours", 3))})
ds["windspeed"] = np.sqrt((ds["U10"]**2 + ds["V10"]**2))
ds["wind_direction"] = (np.rad2deg(np.arctan(ds["V10"] / ds["U10"])) * 2)

In [ ]:
ds["windspeed"][0].plot()

In [ ]:
ds["wind_direction"][0].plot()

In [ ]:
ds["T2"][0].plot()

In [ ]:
timestep = 0
lats = ds.coords['XLAT'][timestep]  # extract/copy the data
lons = ds.coords['XLONG'][timestep]
temp = ds.variables['T2'][timestep] - 273

fig, axs = plt.subplots(figsize=(8, 7), nrows=2,ncols=1,gridspec_kw={'height_ratios': [20,1.5]},constrained_layout=True)
pcm=axs[0].pcolormesh(lons,lats,temp,cmap='viridis')
cbar=fig.colorbar(pcm,cax=axs[1], extend='both', orientation='horizontal')
cbar.set_label('Temperature$')

In [ ]:
timestep = 0
lats = ds.coords['XLAT'][timestep]  # extract/copy the data
lons = ds.coords['XLONG'][timestep]
temp = ds.variables['windspeed'][timestep]

fig, axs = plt.subplots(figsize=(8, 7), nrows=2,ncols=1,gridspec_kw={'height_ratios': [20,1.5]},constrained_layout=True)
pcm=axs[0].pcolormesh(lons,lats,temp,cmap='viridis')
cbar=fig.colorbar(pcm,cax=axs[1], extend='both', orientation='horizontal')
cbar.set_label('Windspeed')

In [ ]:
# I want to find the speed at a certain lat/lon point.
lat = 21.22
lon = -122.68
def select_point(ds, lat, lon):
    # First, find the index of the grid point nearest a specific lat/lon.   
    abslat = np.abs(ds.XLAT[0]-lat)
    abslon = np.abs(ds.XLONG[0]-lon)
    c = np.maximum(abslon, abslat)

    ([xloc], [yloc]) = np.where(c == np.min(c))

    # Now I can use that index location to get the values at the x/y diminsion
    return ds.sel(south_north=xloc, west_east=yloc)

In [ ]:
# I want to find the speed at a certain lat/lon point.
lat = 21.22
lon = -122.68
point_ds = select_point(ds, lat, lon)


In [ ]:
point_ds["T2"].plot()

In [ ]:
ds

# Upload file to api
data format:

In [ ]:
# I want to find the speed at a certain lat/lon point.
lat = 21.22
lon = -122.68
point_ds = select_point(ds, lat, lon)

In [ ]:
point_ds.attrs["SIMULATION_START_DATE"]

In [ ]:
point_ds.keys()

In [ ]:
variables = list(point_ds.keys())
variables.sort()
variables

In [ ]:
ts = pd.to_datetime(str(time.values)).strftime('%Y.%m.%d')

In [ ]:
"2022-09-01T04:00:00Z"

In [ ]:
json_arr = []

forecast_timestamp = pd.to_datetime(
    point_ds.attrs["SIMULATION_START_DATE"], 
    format="%Y-%m-%d_%H:%M:%S"
).strftime('%Y-%m-%dT%H:%M:%SZ')


for i, time in enumerate(point_ds.Time):
    json_arr.append({
        "Location": "Amsterdam",
        "Timestamp": pd.to_datetime(str(time.values)).strftime('%Y-%m-%dT%H:%M:%SZ'),
        "Forecast_timestamp": forecast_timestamp,
        "windspeed": point_ds["windspeed"][i].item(),
        "wind_direction": point_ds["wind_direction"][i].item(),
        "temperature": point_ds["T2"][i].item(),
        "rain": point_ds["RAINNC"][i].item()
    })
json_arr

In [ ]:
r = requests.post("http://localhost:8080/v1/weather", json=json_arr)
r.raise_for_status()

In [ ]:
point_ds["T2"]

In [ ]:
point_ds["windspeed"]

In [ ]:
point_ds["wind_direction"]

In [ ]:
 # 'RAINC',
 # 'RAINNC',
 # 'RAINSH',
    
point_ds["RAINNC"]